# Differential Gene Expression Notebook
Welcome to the Differential Gene Expression workflow for the SSRP workshop. This notebook will cover step-by-step in greater details the downstream RNA-seq analyses. 

As this is built to operate on our [Binderhub](https://binderhub.readthedocs.io/en/latest/index.html), the environment has been preconfigured with appropriate software installed. These softwares will be called as needed. To view them, please see the environment file within the binder directory. If you try to run this notebook on a local machine without properly installing each software package, it will not operate correctly.

This is an attenuated workflow focusing on just the downstream analyses. For a walkthrough that includes the upstream processing, you can see the [full workshop here.](https://github.com/RU-MaGIC-Classes/SSRP_Workshop) 

As this is focused on just the downstream, this will be exclusively in R for ease. 

### Analyses steps
Since we will be skipping the first few, this is a summary of what has/will be performed
- Primary Analysis: Initial demultiplexing and conversion from raw image files to .fastq files. This should include initial sequencing QC. 
- Secondary Analysis: The alignment steps and associated QC. This should include things like removal of garbage reads, ambiguous bases, and for RNAseq quantification of things like ribosomal content. 
- Tertiary analysis: The steps we will start with here. The alignment and feature generation was performed upstream and we start with those files to perform the actual differential gene expression comparisons. 
- Quartenary analysis: We will also perform some of these, this includes visualization and things like pathway analysis, but are only limited by your biological question's contraints. 

## Tertiary analysis start
Now that we have determined we have a quality mapping and we have our table of gene counts, we are ready for the next step- differential gene expression analysis. One of the most prolificly utilized is [DESeq2.](https://bioconductor.org/packages/release/bioc/html/DESeq2.html)

Specifically for RNA-seq experiments like this, we need to take into account several aspects of the biology at play for the statistics. Primarily, a standard t-test assumes a normal distribution. Like a coin flip- If you flip a coin 50 times per test and do 50 test, you would expect the brunt of the output to be near the 25 heads/25 tails, with it being less and less likely to get 1 heads/49 tails or 1 tails/49 heads. That would be a normal distribution. With RNA-seq, we expect to see thousands of "tests" (genes) with many of them having few coin flips (reads) and then a select few having a larger portion of reads. This is more comparable to the lottery, many many will play, some will win a little bit, and very few will hit the jackpot. This type of distribution is usually called a Poisson distribution. RNA-seq goes even a bit further and no longer assumes that the mean and variance will be the same, and instead assumes that the variance is independent of the mean. This is called negative binomial distribution, and as you can see below fits an RNA-seq distribution.

![https://biohpc.cornell.edu/doc/RNA-Seq-2019-Lecture3.pdf](img/nb_mean_var.png)

You also have to consider how to normalize your data. This is critical in RNA-seq data for the following effects:

- Significance due to different sequencing depth. If Sample A is twice as deep as Sample B, well then the genes will look twice as expressed
- Significance due to gene size. If Gene X is twice as long as Gene Y, you would expect more reads to map to Gene X since more of the original RNA would come from the larger gene, and therefore Gene X will look more expressed than Gene Y
- Significance due to RNA depth/composition. There will be several genes that are the most expressed based on the negative binomial distribution. These "super signals" will reduce the visibility of other signals. I always think of a car with those stupid bright HID LED lights driving at night looks like the light of the sun, and then the next car with normal headlights looks like its lights arent even on. Similar concept.

With DESeq2, we fortunately can take into account these normalizations which we will go into a bit more below.

### Load the required libraries
This is where we load all the various libraries that we will be needing for the analyses. 

In [ ]:
library(DESeq2)
library(tidyr)
library(tidyverse)
library(dplyr)
library('pheatmap')
library(RColorBrewer)
library(ggplot2)
library('PCAtools')
library(plotly)
library(EnhancedVolcano)
library(scatterplot3d)

library(clusterProfiler)
library(msigdbr)
library(stringr)
library(enrichplot)
#These below arent true libraries, but are specific parameters we will use for the quartenary analysis. 
organism='org.Mm.eg.db'
korg='mmu'
msig_org='Mus musculus'

### Import the appropriate files
Now that the libraries are loaded, we need to import the files with the data that comes from secondary analysis. For RNA-seq, this can include a raw hit count file like what we have here. Other outputs could include things like Salmon TPMs etc. 

You also must have a metadata file. Metadata is all of the associated data for each sample such as group name, maybe some clinical features etc. 

#### The input hit count file
This is where we load first the hit count file. This could be a comma or tab delimited file (csv or tsv)
So you may need to change the sep parameter as needed. 

In [ ]:
filecontents <- read.csv('./input_data/merged_gene_counts.txt', header=T, sep='\t', row.names=1)

#In this case, the file has both the ensembl ID and gene symbol. We only want to use the ensembl ID as it is unqiue for each gene
#We also then order the columns for parsing later
counts <- filecontents[, -c(1)]
counts <- counts[,order(names(counts))]

##### Creating a table to parse the ensembl ID to gene symbol
As mentioned above, the input data has both the ensembl ID and gene symbol. Ensembl IDs are not exactly human readable, but they are unique for each region and each region has one. So we like to use those for most functions. Building a quick table that keeps the ensembl ID paired with each gene symbol is extremely useful to parse that apart later and make easier visualizations. 

In [ ]:
genetable <- subset(filecontents, select=c(1))
names(genetable)[1] <- 'Geneid'
genetable$Gene <- rownames(genetable)

#### The metadata file
This file should contain all of the different non-RNA based data points. At its most simple form, it should have the sample names and what groupings you have. In more complication experiments this will also include clinical features, batching information, etc. 

In [ ]:
filecontents = read.csv('./input_data/meta.csv', header=T, sep=',', row.names=1)

#Now since we have already ordered out hit count data, we can now order the metadata to keep everything aligned. 
sampletable <- filecontents[order(row.names(filecontents)),]
countdata <- as.matrix(counts)[, colnames(counts) %in% rownames(sampletable)]

### Performing DESeq
Now that you've loaded your data and got it inputted correctly, you need to consider how DESeq2 performs differential gene expression analysis and the normalization mentioned above. You will need to:

- Model the raw counts for each gene
    - Estimate size factors. Normalizes the data using a median of ratios method
    - Estimate gene-wise dispersion. The spread of variability in the data.
    - Fit and shrink to gene-wise dispersion. For the RNA-depth/composition comparison, assumes that similar expression levels have similar dispersion and can be shrunk together to the generalized fit.
    - Generalized Linear Model fit for each gene
- Shrink log2 fold changes if applicable. This wont change the output stats, but will reduce the variance of the samples to the means of the gene, and can make for different visualizations.
- Statistically test for differential gene expression (more detail below)

As briefly touched on above, this will help you normalize the data to take into account sequencing depth, gene size, and RNA depth/composition. All together, this starts to look like:

![https://github.com/hbctraining/DGE_workshop_salmon/blob/master/img/deseq_dispersion2.png](img/deseq_dispersion2.png)

You can see how it brings the normalized counts onto a properly fit dispersion pattern.

We also need to define our design. In this case, it is set up to be as simple as possible. This is always the best way to design an experiment- reduce the variables as much as you can so that the differences you see are derived from the question at hand. In this case, we already have this defined in our metadata as the Group column, and then we build the DESeq Matrix for processing. 

In [ ]:
sampletable$condition <- factor(sampletable$Group)
dds <- DESeqDataSetFromMatrix(countData=countdata, colData=sampletable, design=~ condition)

In the end one of the great things about DESeq2, is that it covers each of the aspects mentioned above in terms of normalizing and if you manually want to adjust them you can, but it will automatically perform each of these steps with a simple command:

In [ ]:
dds <- DESeq(dds)

Once that has completed, we can now look at the dispersion for our samples to make sure it looks appropriate. 

In [ ]:
plotDispEsts(dds, main="Dispersion plot")

### Sample clustering and relatedness
Now at this step it also is good practice to look at how related samples are, which can also act as a mild QC to check that there werent any obvious sample mix ups. In this case, we will do a transformation on the data for visualization purposes, and then we will do a sample relationship distance matrix and PCA plot.

Both distance matrices and PCA plots are incredibly useful tools. PCA is primarily for dimensionality reduction- reducing your thousands of RNAseq genes to a smaller number of vectors that contain the variance. This can be more agnostic, as more similar samples will group together and more dissimilar samples will be farther apart. For PCA I like to use [PCATools](https://bioconductor.org/packages/release/bioc/html/PCAtools.html) for its ease of use as well as ability to correlate clinical data to eigenvectors (if applicable). 
Distance matrices can be a bit different since you are measuring distances between samples in either an agglomerative or divisive fashion. Generally for RNAseq we will use agglomerative hierarchal clustering as done below- based on euclidean distance of the samples. 

To look at this, we can use a variety of values as input. You can start with just the raw counts, but that is not advised since there is no correction in place yet. Usually it is better practice to at least use the partially normalized counts as done here, which pulls the normalized hit counts from the DESeq object. 

In [ ]:
p <- pca(counts(dds, normalized=TRUE), metadata=sampletable, removeVar=0.1)
uncorrected_pca <- biplot(p, colby='Group',
            legendPosition='top', legendLabSize=16, legendIconSize=8.0,
            pointSize=6,
            labSize=5)
print(uncorrected_pca)

Though this partially takes in to account the normalization, it does not fully use the data from the DESeq model. To pull out that, you need either the Variance Stabilizing Transformation (vst) or the regularized log (rlog) values. Both will produce log2 values of the data, VST is the faster option and generally is no different than rlog. rlog is superior when there are variant size factors between your groups- ie one group has 8 samples and the other has 4. Here we will use VST since our groups are all equal. 

In [ ]:
#Extract the VST data
vst <- vst(dds,blind=TRUE)

#Pull the assay data as a matrix. 
mat <- assay(vst)

## These next two lines Ive commented out as examples. This is one way you could perform a batch correction. 
#mm <- model.matrix(~condition, colData(vst))
#mat <- limma::removeBatchEffect(mat, vst$batch, design=mm)

#And now we are going to plot out the PCA plot
p <- pca(mat, metadata = sampletable, removeVar = 0.1)
pca_plot <- biplot(p, colby='Group',
               shape='batch',
            legendPosition='top', legendLabSize=16, legendIconSize=8.0,
            pointSize=6,
            labSize=5)
print(pca_plot)

#### Just as some added fun...
You can also take the PCA plots and make 3D plots out of them, if you ever have additional parameters you need to show. This will not be annotated- try annotating it yourself for some practice to see if you understand the various facets!

In [ ]:
par(mar=c(4,4,4,4), cex=1.0, cex.main=0.8, cex.axis=0.8)
colors <- c("#4169E1", "#FF0000")
groups <- colors[as.numeric(sampletable$Group)]

shapes = c(15,16) 
shapes <- shapes[as.numeric(sampletable$Infected)]

scatterplot3d(p$rotated[,1:3], angle=-40, main="", color=groups, pch=shapes, 
              xlab=paste("PC1, ", round(project.pca.proportionvariances[1], 2), "%"), 
              ylab=paste("PC2, ", round(project.pca.proportionvariances[2], 2), "%"), 
              zlab=paste("PC3, ", round(project.pca.proportionvariances[3], 2), "%"), grid=FALSE, box=FALSE, cex.symbols=2)

source('http://www.sthda.com/sthda/RDoc/functions/addgrids3d.r')
addgrids3d(project.pca$x[,1:3], grid = c("xy", "xz", "yz"))

source('http://www.sthda.com/sthda/RDoc/functions/addgrids3d.r')
addgrids3d(project.pca$x[,1:3], grid = c("xy", "xz", "yz"))


legend("right", levels(sampleTable$Group),
      col =  colors, pch = c(15,16))



As mentioned above, the other way to visualize sample relatedness is hierarchal clustering of samples. Here will will first generate the distance matrix, then plot it out in a heatmap format.

In [ ]:
sampleDists <- as.matrix(dist(t(mat))))
anno_cols <- which(names(sampletable)=='Group')
annotations=metatemp[,anno_cols, drop=F]
colors <- colorRampPalette(rev(brewer.pal(9, "Greys")))(30)

distance_matrix_plot <- pheatmap(sampleDists, annotation_row=annotations, annotation_col=annotations, col=colors,
    fontsize_col=10, fontsize_row=10,angle_col=45
)
print(distance_matrix_plot)

So far, these samples are looking pretty good! The different populations should be grouping together, and should be apart from the other groupings. 

### Actually performing differential gene comparisons
This is where we actually pull out the comparisons and can say "Gene X is significantly different in Group A over Group B!"
For the statistics, DESeq2 traditionally uses the Wald test. The Wald test specifically takes into account the dispersion model distance and the negative binomial distribution that we experience with RNA-seq. We then assume our null hypothesis is that there is no differential gene expression between the groups, and p is small, we can reject the null and say the gene is differentially expressed. One last thing to consider with the p-value is multiple testing correction. Traditionally p < 0.05 is considered significant (95% chance it is not by random chance), but when you are looking at thousands of genes, you'll still end up with 5% of them significant off predicted chance. One of the best corrections to use called False-Discovery Rate correction (FDR), which is the Benjamini and Hochberg method. This corrects the p-value based upon ranking the genes and accounting for the number of genes in the dataset. If you want to be even more conservative, you can use a bonferroni correction, but here we use FDR.

In [ ]:
#I have a particular loop to do this- just because often times I set this up and have multiple comparisons to run. 
#I just implement my loop regardless just cause its easier for me on a code base. 
#But you set up the comparisons in the "compares" list. 

compares <- list(
    list('Cell1Bug','Control'),
    list('Cell1Fungi','Control'),
    list('Cell2Bug','Control'),
    list('Cell2Fungi','Control')
)

#Make a quick directory to store the output files
dir.create('./DEG/')#DEG stands for differentially expressed genes

#For each comparison I then pull out the results of the deseq based on the comparisons, then perform a logfc shrink. 
#This doesnt change what is significant or not, but makes it more even for plotting purposes. 
for(i in compares){
    #Pull the results from the dataset and the appropriate contrasts
    out_res <- results(dds, contrast=c('condition',i[[1]], i[[2]]), alpha=0.05)
    #Perform the lfc shrinkage. 
    shrink <- lfcShrink(dds, contrast=c('condition',i[[1]], i[[2]]),res=out_res, type="normal")
    comp_out <- shrink[order(shrink$padj),]
    #Parse back the normalized values. Again you can use either the log2 values from VST or normalized hit counts
    comp_out <- merge(as.data.frame(comp_out), as.data.frame(mat),
    by='row.names', sort=FALSE)
    names(comp_out)[1] <- 'Gene'
  
    #Now this is where that table with the ensembl IDs paired with the gene symbols comes in handy
    #You can now add that information back in so you are looking at genes that make a bit more sense
    DataIn <- merge(as.data.frame(comp_out), as.data.frame(genetable), by="Gene", sort=FALSE)
    DataIn <- DataIn %>% relocate(GeneID, .after = Gene)
    #Write it out as a nice file
    write.csv(DataIn, file=paste('./DEG/',i[[1]],'_vs_',i[[2]],'.csv', sep=''))
    #And save the comparison in R for us to use later
    assign(paste(i[[1]],'_vs_',i[[2]], sep=''), DataIn)
}

Now you have your statistics performed on your data and you can start looking at what biologically is going on!

## Quartenary analysis start
Now the last stages of analysis are also the most open ended- and this is considered quartenary analysis. This is where you now start to connect all the hard work you've done into some biological conclusions. In a lot of cases you'll already have some aspect you are focusing in on, maybe like the role of TH1 genes in your immune reponse to drug X or something, but other times you are purely on a fishing expedition. So here Ill leave it a bit more open ended- how would you start to explore this data?

You could look at specific genes of interest, look at just the top genes, start plotting them out in different fashions. Ill give a few examples below, but then start to explore the data and come up with some biological conclusions.

### Volcano Plots
The first place I like to start is looking at the direct group comparisons. What is different in my condition of choice to control?? Of course, you are going to want to visualize this. You can do this with standard packages, but why reinvent the wheel when there are great packages already to make it easier. In this case, I use [EnhancedVolcano](https://bioconductor.org/packages/3.13/bioc/html/EnhancedVolcano.html) as a visualization aid. 

For most of these plots as we progress, I will generate one as an example. I highly recommend trying these out and generating the plots for other samples/genes etc. 

Volcano plots are a staple in RNA-seq analysis. These are generally used to look at the spread of differentially expressed genes between two samples. They are usually structured to show the high count of insignificant genes in on the bottom, then the more significant and alternatively regulated genes exploding out- looking a bit like a [plinian volcanic eruption](https://www.youtube.com/watch?v=VBTAcACmcgo) (hence the name- volcano plots).

In [ ]:
#For the specific parameters to tweak, I would recommend reading the packages documentation. 
cell1bug_volcano <- EnhancedVolcano(Cell1Bug_vs_Control,
                                   lab=as.character(Cell1Bug_vs_Control$Geneid),
                                   title='Cell1Bug Infected versus Control',
                                   x='log2FoldChange',
                                   y='padj',
                                   legendPosition='bottom',
                                   legendLabels=c('Not significant','Log2FC','padj','padj and Log2FC'),
                                   pCutoff=0.05,
                                   FCcutoff=1)

print(cell1bug_volcano)

Like I said above, I highly recommend fiddling with it a bit and doing the plots for the other comparisons as well here. Just add the cells!

### Heatmaps
Heatmaps can come in many flavors, sometimes you want to show just significant genes, the most variant genes between samples, maybe a list from a specific pathway etc. You also can do things like cluster the genes and samples as we did above. 

Here we will do a quick heatmap just looking at the top 250 most variant genes across all samples. One other note, colors used for heatmaps is constantly changing. The canonical colors are based on microarrays, with green being down and red being up. Personally that color combination drives me nuts (green means go, red is bad), so I always use something else. A very common coloration now is a red/blue combo, and also a magma gradient. There is also an increasing trend in colorblind friendly schemes, which is always great to use. 

In [ ]:
#Determine what the top genes are
topVarianceGenes <- head(order(rowVars(mat), decreasing=T), 250)

#Set up a nice color palette.
my_colors = brewer.pal(n = 11, name = "RdBu")
    my_colors = colorRampPalette(my_colors)(50)
    my_colors = rev(my_colors)

#Assign some quick column annotations
mat_col <- data.frame(group=sampletable$Group)
rownames(mat_col) <- colnames(mat)

#Use the pheatmap package to generate the plot. In this case we will be clustering both the columns and rows
heatmap_plot <- pheatmap(mat[topVarianceGenes,], cluster_rows=TRUE, color=my_colors,
    show_rownames=FALSE, cluster_cols=TRUE, scale='row', fontsize=20, annotation_col=mat_col)

return(heatmap_plot)

As with the volcano plots, I recommend making a few more heatmaps for practice. A good idea would be to take a genelist and use those to make a specific heatmap!

### Cross comparison comparisons!
Beyond just the cross comparisons themselves, it is common to then want to compare those comparisons. There are a variety of ways to do this, but most commonly are venn diagrams and upset plots. 

#### Venn Diagrams
Venn diagrams are a classic way to show overlap and uniqueness between comparisons. These are very powerful visualizations, but can become messy as the number of samples increase. Realistically two or three samples is viable, four is on the edge, five is very messy, and six+ should not be done. In that case, things should divert to upset plots. 

In this case we will do all of the comparisons that we made earlier! 

In [ ]:
#Make a list of the earlier comparisons
compares <- list(
    list("Cell1Bug", Cell1Bug_vs_Control),
    list("Cell1Fungi", Cell1Fungi_vs_Control),
    list("Cell2Bug", Cell2Bug_vs_Control),
    list("Cell2Fungi", Cell2Fungi_vs_Control)    
)

# Create an empty list to fill up as we go
vennlist <- list()
venncolors <- list('#0000ff','#00ff00','#ff0000','#f0000f')

#A nice little loop to go through each of the comparisons we set up above, and then pull the list of significant genes. 
for (DataSet in compares){
    sig_genes <- c() #The vector of significant genes to fill up as we go
    DataIn <- as.data.frame(DataSet[2]) #Pull just the output comparison of the data
    DataIn <- subset(DataIn, padj < 0.05) #And select the significant genes only
    
    #And pull the gene names just as we did before
    original_gene_list <- DataIn$log2FoldChange
    names(original_gene_list) <- DataIn$Gene
    gene_list<-na.omit(original_gene_list)
    gene_list = sort(gene_list, decreasing = TRUE)
    genes <- names(gene_list)
    
    for (item in genes){
        sig_genes <- c(sig_genes, item)
    }
    
    #Get just the unique names
    sig_genes <- unique(sig_genes) 
    #And put them in to the big list
    vennlist[[paste(DataSet[1])]] <- sig_genes
}

#So now that the venn diagram list is built we can use it to make the actual diagram!
fillcolors <- unlist(venncolors)

vennplot <- venn.diagram(x=vennlist,
    fill=fillcolors,
    lwd=5,
    lty=1,
    cat.cex=4,
    cex=5,
    alpha=0.5,
    filename=NULL
)

print(grid.draw(vennplot))

You can focus this further by selecting only up or downregulated genes as well. 

#### UpSet Plots

In [ ]:
#Since we have already built the list above, we can reuse it here. 
upset(fromList(vennlist), order.by='freq',
    text.scale=c(2, 2, 2,
        2, 3, 2),
    point.size=6, 
    line.size=4,
    mainbar.y.label='Gene intersections', sets.x.label='Significant genes',
    )

Likewise to venn diagrams, you can focus it by selecting up or downregulated genes only as well. 

### Gene Ontology/Pathway Analysis
Having individual genes is great, but the next step is nearly always related to groups of genes and how they could be inter-related. For the major organisms like Human/mouse/rat, there are a plethora of tools available. Things like and are commonly purchased for large scale licenses, with both available here at Rutgers. Regardless, there are also gene ontologies for a much wider berth of organisms, as well as custom gene ontologies that can be manually curated and broad pan-bacterial ontologies. 

Here at Rutgers, we do have licenses for both [Ingenuity Pathway Analysis (IPA)](https://digitalinsights.qiagen.com/products-overview/discovery-insights-portfolio/analysis-and-visualization/qiagen-ipa/) and [Advaita's iPathwayGuide](https://advaitabio.com/ipathwayguide/) as proprietary options. Both have their plusses and minuses, we could go in to this at depth if anyone wishes but really it comes down to which you are more comfortable with and if you really want their specially curated pathways. In reality, its often just as strong (and massively cheaper) to use open source alternatives. 

To that, we will use [clusterProfiler](https://bioconductor.org/packages/release/bioc/html/clusterProfiler.html). This is a particularly powerful tool as it allows us to use multiple methods as well as multiple databases for enrichment. 

Enrichment is usually done either as over representation analysis (ORA) or functional class scoring (FCS), and we will explore both here. 

#### Over Representation Analysis
Most proprietary platforms using ORA, occasionally with slight modifications. To perform ORA, you are selecting which features in your data you are looking at, and then comparing them to a database of ontologies/pathways. In this, your selection criteria is critical as it directly works into the significance calculations. Generally, the best cutoffs to use is just a significance in terms of the gene vs control (padj < 0.05) and then directionality (up or down regulated). Anything more can get a bit dicey. 

ORA is calculated by taking the number of genes in your input list that are in the pathway in question over the size of your input list. If that ratio is higher or lower than expected based on the number of genes in that pathway compared to the total number of genes, you woud have enrichment or deenrichment. 

In this case, Ill be using the standard [Gene Ontology database](http://geneontology.org/) as the reference. This is a community curated list of pathways essentially, and is split in to biological processes, cellular component, and molecular function. 

In [ ]:
#Here I will use just one sample again.First by subsetting it to just the significant genes
cell1bug <- subset(Cell1Bug_vs_Control, padj < 0.05)

#If I also wanted to filter for just the upregulated genes I could add something like this
#cell1bug_ora <- subset(cell1bug_ora, log2FoldChange >0)
#But in this case I want all the signficant genes so I wont

#Now I do some funkiness to get the list of genes
original_gene_list <- cell1bug$log2FoldChange
names(original_gene_list) <- cell1bug$Gene
gene_list <- na.omit(original_gene_list)
gene_list <- sort(gene_list, decreasing=TRUE)
cell1bug_ora_genes <- names(gene_list)

#And do the actual ORA and plot
cell1bug_ora <- enrichGO(gene=cell1bug_ora_genes,
                        ont='BP',
                        keyType='ENSEMBL',
                        pvalueCutoff=0.05,
                        OrgDb=organism,
                        pAdjustMethod='none')
cell1bug_ora_plot <- dotplot(cell1bug_ora, showCategory=10, font.size=10) + scale_y_discrete(labels=function(x) str_wrap(x, width=40))
#Make a quick directory to store the output files
dir.create('./Pathways/')
#Make a table for the full output, likewise you could read it here with just head(cell1bug_ora)         
write.csv(cell1bug_ora, file='./Pathways/Cell1Bug_ORA_GO_totalsig.csv')
print(cell1bug_ora_plot)

You can try exploring other databases as well with clusterProfiler. Included are things like [KEGG](https://www.genome.jp/kegg/), [WikiPathways](https://www.wikipathways.org/), [Reactome](http://bioconductor.org/packages/ReactomePA), etc. 

#### Functional Class Scoring
Functional class scoring is also commonly called gene set enrichement analysis (GSEA), which is also its most famous implementation by the Broad. Unlike ORA, FCS takes in to account all of the genes in the dataset, but is predicated on a ranking system instead. Generally this can be ranked by p-value or log2FoldChange, but also can include things like the Wald statistic which would factor both. The concept behind FCS is that lots of small changes that are technically insignficant may aggregate to form a signficant shift in an overall pathway, and thereby a phenotypic change. 

For this case, Ill use a different database to demonstrate the flexibility of clusterProfiler and how you would convert to a different ID type than ensembl. 

In [ ]:
#Ill start it as above to keep things akin, but note how there is no filtering first. 
original_gene_list <- Cell1Bug_vs_Control$log2FoldChange
names(original_gene_list) <- Cell1Bug_vs_Control$Gene

#Now we use the bitr function that is built in to clusterProfile to conver to EntrezIDs, and sort them for FCS
ids<-bitr(names(original_gene_list), fromType = "ENSEMBL", toType = "ENTREZID", OrgDb=organism)
dedup_ids = ids[!duplicated(ids[c("ENSEMBL")]),]
fcs_df = Cell1Bug_vs_Control[Cell1Bug_vs_Control$Gene %in% dedup_ids$ENSEMBL,]
fcs_df$Entrez = dedup_ids$ENTREZID
cell1bug_kegg_gene_list <- fcs_df$log2FoldChange
names(cell1bug_kegg_gene_list) <- fcs_df$Entrez
cell1bug_kegg_gene_list<-na.omit(kegg_gene_list)
cell1bug_kegg_gene_list = sort(cell1bug_kegg_gene_list, decreasing = TRUE)

cell1bug_fcs <- gseKEGG(geneList=cell1bug_kegg_gene_list,
                       organism=korg,
                       nPerm=as.numeric(1000),
                       pvalueCutoff=0.05,
                       minGSSize=3,
                       maxGSSize=800,
                       verbose=FALSE,
                       pAdjustMethod='none')

cell1bug_fcs_plot <- dotplot(cell1bug_fcs, showCategory=10, font.size=10) + scale_y_discrete(labels=function(x) str_wrap(x, width=40))
#Make a quick directory to store the output files
dir.create('./Pathways/')
#Make a table for the full output, likewise you could read it here with just head(cell1bug_ora)         
write.csv(cell1bug_fcs, file='./Pathways/Cell1Bug_FCS_KEGG.csv')
print(cell1bug_fcs_plot)

Again, could be fun to then test this out with different databases and tweaking some of the parameters to best fit your data. With FCS in particular, you can also plot out the classic GSEA plot. 

In [ ]:
#In this case we are just going to take the top pathway in the file
cell1bug_fcsgsea_plot <- gseaplot(cell1bug_fcs, geneSetID=as.numeric(1), by='runningScore',title='First Pathway GSEA plot')
print(cell1bug_fcsgsea_plot)

### Single gene plots
Single genes plots are a great way to show something that you have focused in on. Sometimes a whole RNA-seq experiment is focused for one gene in the end, and this is a great way to show it across groups. 

In [ ]:
index <- which(genetable=='Actb')
ens_gene_select <- genetable[as.numeric(index), "Gene"]

filtered <- t(log2((counts(dds[ens_gene_select, ], normalized=TRUE, replaced=FALSE)+.5))) %>%
       merge(colData(dds), ., by="row.names") %>%
       gather(gene, expression, (ncol(.)-length(ens_gene_select)+1):ncol(.))

p <- ggplot(filtered, aes_string('Group', "expression", fill='Group'))
p <- p + geom_boxplot() + facet_wrap(~gene, scales="free_y")

p <- p + xlab(' ') + ylab('Normalized log2 Expression') + theme(
  plot.margin = unit(c(1,1,1,1), "cm"),
  axis.text.x = element_text(angle = as.numeric(45),size=as.numeric(5)),
  axis.text.y = element_text(size=as.numeric(5)),
  plot.title=element_text(size=as.numeric(10)),
  legend.position='bottom') + ggtitle('Actb gene plot')

You could also plot these via violin plots as an alternative to box plots. 